In [1]:
import pandas as pd
import numpy as np

nonsquare_df1 = pd.read_csv('./spmm-nonsquare-traindata-v2-doptimal-900.csv')
nonsquare_df2 = pd.read_csv('./spmm-nonsquare-traindata-v2-doptimal-100.csv')

In [2]:
from sklearn.model_selection import train_test_split

# 900개
X_train = nonsquare_df1[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd']] 
y_train = nonsquare_df1['bz_smdm_median']
# 100개
X_test = nonsquare_df2[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd']] 
y_test = nonsquare_df2['bz_smdm_median']

In [3]:
# 데이터 정규화
mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std

X_test -= mean
X_test /= std

In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [5]:
from sklearn.metrics import mean_squared_error

def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def rmse(y_true, y_pred):
    rmse = np.sqrt(np.mean(np.square(y_pred - y_true))) 
    return rmse

def custom_scoring(y_test, y_pred):

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return mape

In [6]:
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import KFold
# from sklearn.metrics import make_scorer

# grid = {
#     'hidden_layer_sizes' : [],
#     'learning_rate_init' : [0.0005,0.001,0.005]
# }

# regressor = MLPRegressor(max_iter=5000,early_stopping=True)

# regressor_grid = GridSearchCV(regressor, 
#                               param_grid = grid, 
#                               scoring = make_scorer(custom_scoring, greater_is_better=False),
#                               n_jobs=-1,
#                               verbose=3) 

# regressor_grid.fit(X_train, y_train)

# print("최고의 파라미터 :", regressor_grid.best_params_)
# print("최고 평균 정확도 : {}".format(regressor_grid.best_score_)) 

In [7]:
# from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import KFold
# from sklearn.metrics import make_scorer

# kf = KFold(random_state=30,
#            n_splits=10,
#            shuffle=True,
#           )

# grid = {
#     'hidden_layer_sizes' : [(16,),(32,),(64,)],
#     'alpha' : [0.0001,0.0005,0.001, 0.005, 0.01],
#     'learning_rate_init' : [0.005, 0.01, 0.05, 0.1],
# }

# regressor = MLPRegressor(max_iter=5000)

# regressor_grid = GridSearchCV(regressor, 
#                               param_grid = grid, 
#                               scoring = make_scorer(custom_scoring, greater_is_better=False),
#                               cv=kf, 
#                               n_jobs=-1,
#                               verbose=3) 

# regressor_grid.fit(X_train, y_train)

# print("최고의 파라미터 :", regressor_grid.best_params_)
# print("최고 평균 정확도 : {}".format(regressor_grid.best_score_))

In [ ]:
from sklearn.neural_network import MLPRegressor

regressor = MLPRegressor(hidden_layer_sizes=(128, 512, 1024, 2048, 1024, 512, 128,), 
                         max_iter=5000,
                         #activation='relu',
                         #alpha=0.0001,
                         #learning_rate_init=0.001,
                         early_stopping=False
                         )

regressor.fit(X_train, y_train)

y_pred_train = regressor.predict(X_train)
y_pred = regressor.predict(X_test)

In [ ]:
# 훈련 자체 에러율
print("--------train 에러율--------")
print("rmse : {}".format(rmse(y_train,y_pred_train)))
print("mape : {}".format(mean_absolute_percentage_error(y_train, y_pred_train)))

# 테스트 에러율
print("--------test 에러율--------")
print("rmse : {}".format(rmse(y_test,y_pred)))
print("mape : {}".format(mean_absolute_percentage_error(y_test,y_pred)))